## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override = True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succinctly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(name = "Search agent", 
                     instructions = INSTRUCTIONS, 
                     tools = [WebSearchTool(search_context_size = "low")], 
                     model = "gpt-4o-mini", 
                     model_settings = ModelSettings(tool_choice = "required"))

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, enhancing the development and deployment of intelligent agents across various applications.

**OpenAI Agents SDK** is a lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. It is compatible with over 100 different large language models (LLMs), offering a provider-agnostic approach. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

**Google's Agent Development Kit (ADK)**, announced in April 2025, is a modular framework that integrates with Google's ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and requires minimal code for efficient development. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

**OutSystems' Agent Workbench**, launched recently, enables enterprises to implement agentic AI to streamline operations and modernize legacy systems. It facilitates the development and coordination of intelligent agents using a low-code approach across various workflows and data sources. ([techradar.com](https://www.techradar.com/pro/outsystems-agent-workbench-reaches-general-availability-helping-enterprises-streamline-operations-through-agentic-ai?utm_source=openai))

**Agent Lightning** is a flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent. It achieves complete decoupling between agent execution and training, allowing seamless integration with existing agents developed via diverse methods. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

**Cognitive Kernel-Pro** is a fully open-source and free multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It systematically investigates the curation of high-quality training data for Agent Foundation Models, focusing on the construction of queries, trajectories, and verifiable answers across key domains. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

**AutoAgent** is a fully-automated and zero-code framework for LLM agents, enabling users to create and deploy LLM agents through natural language alone. It operates as an autonomous Agent Operating System, comprising key components that facilitate efficient and dynamic creation and modification of tools, agents, and workflows without coding requirements or manual intervention. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

**Kruti**, developed by Ola Krutrim and released on June 12, 2025, is a multilingual AI agent and chatbot designed to perform real-world tasks for users, such as booking taxis and ordering food, by integrating directly with various online services. It is notable for its ability to understand and respond in multiple Indian languages. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

**Model Context Protocol (MCP)**, introduced by Anthropic in November 2024, is an open standard framework that standardizes the way AI systems like LLMs integrate and share data with external tools, systems, and data sources. MCP provides a universal interface for reading files, executing functions, and handling contextual prompts. It has been adopted by major AI providers, including OpenAI and Google DeepMind. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

These frameworks represent significant advancements in AI agent development, offering diverse tools and approaches to meet the evolving needs of the industry. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason : str = Field(description = "Your reasoning for why this search is important to the query.")
    query : str = Field(description = "The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches : list[WebSearchItem] = Field(description = "A list of web searches to perform to best answer the query.")

planner_agent = Agent(name = "PlannerAgent", instructions = INSTRUCTIONS, model = "gpt-4o-mini", output_type = WebSearchPlan)

In [6]:
planner_agent

Agent(name='PlannerAgent', instructions='You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output 3 terms to query for.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=<class '__main__.WebSearchPlan'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [7]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find current and emerging AI agent frameworks that are gaining traction in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore comparisons and features of new AI agent technologies released in 2025.', query='top AI agent technologies 2025 review'), WebSearchItem(reason='To investigate industry trends and developments in AI agents related to specific applications for 2025.', query='AI agents trends applications 2025')]


In [8]:
@function_tool
def send_html_email(subject : str, html_body : str) -> Dict[str, str]:
    """
    Sends an HTML-formatted email using the SendGrid API.

    <b>*Parameters*</b>
    - subject (str): The subject line of the email to be sent.
    - html_body (str): The HTML content that will be used as the body of the email.

    <b>*Returns*</b>
    - Dict[str, str]: A dictionary indicating the status of the operation (e.g., {"status": "success"}).

    <b>*Logic*</b>
    1. Initialize the SendGrid client using the required API key, retrieved securely from environment variables.
    2. Define and validate the sender's email address (must be verified in SendGrid).
    3. Define the recipient's email address.
    4. Construct the email content using the provided HTML body with MIME type "text/html".
    5. Assemble the Mail object with sender, recipient, subject, and HTML content.
    6. Send the email using SendGrid's Mail API with a POST request.
    7. Return a status message indicating the outcome ("success" if the operation was completed).
    """

    # Initialize the SendGrid client using the API key from the environment
    sg = sendgrid.SendGridAPIClient(api_key = os.environ.get("SENDGRID_API_KEY"))

    # Set the verified sender email address (must be verified in your SendGrid dashboard)
    from_email = Email(email = "siddharthwolverine@gmail.com")  # Change this to your verified sender

    # Set the recipient email address (the person who will receive the email)
    to_email = To(email = "siddharth13101999singh@gmail.com")  # Change this to your recipient

    # Define the email content with the given HTML body and the correct MIME type
    content = Content(mime_type = "text/html", content = html_body)

    # Create the email object with sender, recipient, subject, and content
    # Note: Using 'plain_text_content' to store HTML here is a mistake in SendGrid API usage.
    # The correct argument in Mail() is 'html_content' for HTML body, not 'plain_text_content'.
    mail = Mail(from_email = from_email, to_emails = to_email, subject = subject, html_content = content).get()

    # Send the email via the SendGrid client
    sg.client.mail.send.post(request_body = mail)

    # Return a success response
    return {"status" : "success"}

In [9]:
send_html_email

FunctionTool(name='send_html_email', description='Sends an HTML-formatted email using the SendGrid API.\n\n<b>*Parameters*</b>\n- subject (str): The subject line of the email to be sent.\n- html_body (str): The HTML content that will be used as the body of the email.\n\n<b>*Returns*</b>\n- Dict[str, str]: A dictionary indicating the status of the operation (e.g., {"status": "success"}).\n\n<b>*Logic*</b>\n1. Initialize the SendGrid client using the required API key, retrieved securely from environment variables.\n2. Define and validate the sender\'s email address (must be verified in SendGrid).\n3. Define the recipient\'s email address.\n4. Construct the email content using the provided HTML body with MIME type "text/html".\n5. Assemble the Mail object with sender, recipient, subject, and HTML content.\n6. Send the email using SendGrid\'s Mail API with a POST request.\n7. Return a status message indicating the outcome ("success" if the operation was completed).', params_json_schema={'p

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(name = "Email agent", instructions = INSTRUCTIONS, tools = [send_html_email], model = "gpt-4o-mini")

In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary : str = Field(description = "A short 2-3 sentence summary of the findings.")
    markdown_report : str = Field(description = "The final report")
    follow_up_questions : list[str] = Field(description = "Suggested topics to research further")

writer_agent = Agent(name = "WriterAgent", instructions = INSTRUCTIONS, model = "gpt-4o-mini", output_type = ReportData)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [12]:
async def plan_searches(query : str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query : {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output


async def search(item : WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


async def perform_searches(search_plan : WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

### The next 2 functions write a report and email it

In [12]:
async def write_report(query : str, search_results : list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results : {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report : ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [13]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>